In [ ]:
def partial_fused_gromov_wasserstein(M, C1, C2, p, q, alpha, m=None, G0=None, loss_fun=‘square_loss’, 
                                     armijo=False, log=False,
                                     verbose=False, numItermax=1000, 
                                     tol=1e-7, stopThr=1e-9, stopThr2=1e-9):
    # 定义一个函数，用于计算两个图之间的部分融合Gromov-Wasserstein距离   
    '''
    M: (250, 250)
    D_A: (250, 250)
    D_B: (250, 250)
    a: (250,)
    b: (250,)
    pi: (250, 250)
    M: 两个图节点特征之间的距离矩阵
    C1: 第一个图的结构矩阵(250, 250)
    C2: 第二个图的结构矩阵(250, 250)
    p: 第一个图的节点分布(250,)
    q: 第二个图的节点分布(250,)
    alpha: 融合系数，控制结构和特征的权重
    m: 部分传输映射（partial transport map）的总质量，表示两个图之间匹配的程度
    G0: 部分传输映射的初始值(250, 250)
    loss_fun: 用于计算距离的损失函数，可以是’square_loss’或’kl_loss’
    armijo: 是否使用Armijo线搜索法来更新步长
    log: 是否记录迭代过程中的误差和损失值
    verbose: 是否打印迭代过程中的信息
    numItermax: 最大迭代次数
    tol: 迭代终止条件之一，当相对误差小于tol时停止迭代
    stopThr: 迭代终止条件之一，当绝对误差小于stopThr时停止迭代
    stopThr2: 迭代终止条件之一，当损失值变化小于stopThr2时停止迭代
    '''
    
    if m is None:
        # m = np.min((np.sum(p), np.sum(q)))
        raise ValueError("Parameter m is not provided.")
    elif m < 0:
        raise ValueError("Problem infeasible. Parameter m should be greater"
                         " than 0.")
    elif m > np.min((np.sum(p), np.sum(q))):
        raise ValueError("Problem infeasible. Parameter m should lower or"
                         " equal to min(|p|_1, |q|_1).")
    # 如果没有给定m参数，则抛出异常 # 
    if G0 is None:
        # 如果没有给定G0参数，则用p和q的外积作为初始值
        G0 = np.outer(p, q)

    nb_dummies = 1 # 定义虚拟节点的数量
    dim_G_extended = (len(p) + nb_dummies, len(q) + nb_dummies) # 定义扩展后的部分传输映射的维度
    q_extended = np.append(q, [(np.sum(p) - m) / nb_dummies] * nb_dummies) # 给q添加虚拟节点，使得其总质量等于m
    p_extended = np.append(p, [(np.sum(q) - m) / nb_dummies] * nb_dummies) # 给p添加虚拟节点，使得其总质量等于m
    cpt = 0 # 初始化计数器
    err = 1 # 初始化误差

    if log:
        # 如果需要记录日志，则创建一个字典log来存储误差和损失值
        log = {'err': [], 'loss': []}
    f_val = fgwloss_partial(alpha, M, C1, C2, G0, loss_fun) # 计算部分传输映射G0对应的部分融合Gromov-Wasserstein损失值f_val
    if verbose:
        # 如果需要打印信息，则打印表头和初始值
        print('{:5s}|{:12s}|{:8s}|{:8s}'.format(
            'It.', 'Loss', 'Relative loss', 'Absolute loss') + '\n' + '-' * 48)
        print('{:5d}|{:8e}|{:8e}|{:8e}'.format(cpt, f_val, 0, 0))
        #print_fgwloss_partial(alpha, M, C1, C2, G0, loss_fun)

    # while err > tol and cpt < numItermax:
    while cpt < numItermax: # 进入迭代循环，直到达到最大迭代次数或满足终止条件之一
        Gprev = np.copy(G0) # 复制G0到Gprev
        old_fval = f_val # 保存旧的损失值old_fval

        gradF = fgwgrad_partial(alpha, M, C1, C2, G0, loss_fun) # 计算部分融合Gromov-Wasserstein梯度gradF
        gradF_emd = np.zeros(dim_G_extended) # 创建一个零矩阵gradF_emd，用于存储扩展后的梯度
        gradF_emd[:len(p), :len(q)] = gradF # 将gradF复制到gradF_emd的对应位置
        gradF_emd[-nb_dummies:, -nb_dummies:] = np.max(gradF) * 1e2 # 将gradF_emd的右下角填充为gradF的最大值乘以一个大的系数，以防止虚拟节点之间的匹配
        gradF_emd = np.asarray(gradF_emd, dtype=np.float64) # 将gradF_emd转换为浮点数类型

        Gc, logemd = ot.lp.emd(p_extended, q_extended, gradF_emd, numItermax=1000000, log=True) # 使用线性规划方法（Earth Mover's Distance）求解gradF_emd对应的最优部分传输映射Gc，并记录相关信息logemd
        if logemd['warning'] is not None:
            # 如果logemd中有警告信息，则抛出异常，提示用户增加虚拟节点的数量
            raise ValueError("Error in the EMD resolution: try to increase the"" number of dummy points")
        G0 = Gc[:len(p), :len(q)] # 更新部分传输映射G0为Gc的对应部分
        f_val = fgwloss_partial(alpha, M, C1, C2, G0, loss_fun) # 计算新的损失值f_val
        if armijo:
            # 如果使用Armijo线搜索法，则根据旧的损失值和梯度调整步长
            pass
        err = np.linalg.norm(G0 - Gprev) / np.linalg.norm(Gprev) # 计算相对误差err
        abs_err = np.linalg.norm(G0 - Gprev) # 计算绝对误差abs_err
        if verbose:
            # 如果需要打印信息，则打印当前迭代次数，损失值，相对误差和绝对误差
            print('{:5d}|{:8e}|{:8e}|{:8e}'.format(cpt, f_val, err, abs_err))
            #print_fgwloss_partial(alpha, M, C1, C2, G0, loss_fun)
        if log:
            # 如果需要记录日志，则将误差和损失值添加到log字典中
            log['err'].append(err)
            log['loss'].append(f_val)
        cpt += 1 # 更新计数器

    return G0, log # 返回部分传输映射G0和日志log（如果有）